<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/File_Utilities_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%writefile GDriveFileUtilities.py
# 08/11/2020-1
from __future__ import absolute_import
from IPython.display import clear_output
import glob, os, pathlib, sys
from os.path import *
from pathlib import Path
from google.colab import drive, files

global C
global fu

class ColorOutput:
    LITE = '\033[96m'; HEADER = '\033[95m'; BLUE = '\033[94m';
    WARNING = '\033[93m'; GREEN = '\033[92m'; RED = '\033[91m'; 
    DARK = '\033[90m'; UNDERLINE = '\033[4m';BOLD = '\033[1m'; 
    WHITE = '\033[0m'

C = ColorOutput()

class FileUtilities(ColorOutput):
    __all__ = ['C', 'allFiles', 'allFilesDict', 'cwdPath', 'homePath', 
           'inspectFile', 'listFiles', 'printSystemPaths', 
           'setSystemPaths', 'updateAllFiles', 'mountGoogleDrive',
              'deleteGDriveFile', 'searchGDriveFiles', 'foundFilesDict']

    def __init__(self):
        self.C = ColorOutput
        self.homePath = Path.home()
        self.cwdPath = Path.cwd()
        self.allFiles = glob.glob('**', recursive=True)
        self.allFilesDict = {}
        self.foundFilesDict = {}

    def updateAllFiles(self):
        os.chdir("/content/")
        self.allFiles = glob.glob('**', recursive=True)

    def setSystemPaths(self, addNewPath=None):
        addNewPath = realpath(addNewPath)
        pathList = sys.path.copy()
        if not addNewPath in pathList and exists(addNewPath) and addNewPath != 'None':
            sys.path.append(addNewPath)

    def listFiles(self, silent=False):
        if not silent:
            print(f'{C.BLUE}')
        self.updateAllFiles()
        self.allFiles = sorted(self.allFiles)
        tempDict = self.allFilesDict
        fileCount = 0
        if not silent:
            print(f'{C.BLUE}{self.cwdPath}')
        self.setSystemPaths(addNewPath=self.cwdPath)
        for fil in self.allFiles:
            fullPath = Path(realpath(fil)) # POSIX path
            realPath = realpath(fil) # String path
            if isdir(fullPath) and not realPath.__contains__('__pycache__'):  
                if not silent:
                    print(f'\n{C.BLUE}{fullPath}')
                self.setSystemPaths(addNewPath=realPath) 
            elif isfile(fullPath) and not realPath.__contains__('__pycache__'): 
                fileCount += 1
                if not silent:
                    print(f'{C.WHITE}{fileCount}. {fullPath.name}') 
                tempDict[fileCount] = fullPath
                self.allFilesDict = {}
                self.allFilesDict.update(tempDict)

    def inspectFile(self, selection):
        if selection <= len(self.allFilesDict):  
            try:
                pth = self.allFilesDict[selection]  
                with open(pth, 'r') as openFile:  
                    print(f'# %%writefile {pth.name}\n')  
                    print(openFile.read())
            except:
                print(f'{C.RED}BULLSHIT')

    def printSystemPaths(self):
        pathList = sys.path.copy() 
        pathCount = 0
        for pth in sorted(pathList):
            pathCount += 1
            print(f'{C.WHITE}{pathCount}. {C.BLUE}{pth}')

    def searchGDriveFiles(self):
        os.chdir("/content/")
        print(f'in {C.BLUE}{Path.cwd()}{C.WHITE}')
        searchFilesList = glob.glob('**', recursive=True)
        searchTerm = str(input('search for:' ))

        # searchTerm = '.txt'
        # print(f'search for: {searchTerm}')

        fileCounter = 0
        tempDict = {}
        for fil in sorted(searchFilesList):
            if fil.__contains__(searchTerm) and not fil.__contains__('__pycache__'):
                fullPath = Path(realpath(fil))
                thisPath = realpath(fil)
                if isfile(fullPath):
                    fileCounter += 1
                    tempDict[fileCounter] = fullPath

        self.foundFilesDict = tempDict.copy()
        for item in self.foundFilesDict:
            print(f'{C.WHITE}{item}. {self.foundFilesDict[item].name}') 
            print(f'{C.BLUE}{self.foundFilesDict[item]}\n')

    def deleteGDriveFile(self, deleteFile=None):
        if exists(deleteFile) and isfile(deleteFile):
            clear_output()
            print(f'{C.RED}{deleteFile}{C.WHITE}\n')
            os.remove(deleteFile)
            fu.updateAllFiles()
            
    def mountGoogleDrive(self):
        drive.mount('gdrive', force_remount=True, timeout_ms=600000)

fu = FileUtilities()
fu.mountGoogleDrive()
fu.listFiles(silent=True)
print(f'{C.GREEN}Ok')


In [ ]:
fu.searchGDriveFiles()

In [ ]:
# fu.listFiles()
fu.inspectFile(selection=16)

In [ ]:
def changeDir():
    pathDict = {}
    sysPaths = sys.path.copy()
    counter = 0
    for pth in sorted(sysPaths):
        if pth.__contains__('/content'):
            counter += 1
            pathDict[counter] = pth
            print(f'{C.HEADER}{counter}. {pth}{C.WHITE}')
    choice = int(input('choice: '))
    if choice in range(1, len(pathDict)):
        %cd {pathDict[choice]}
    else: print(f'{C.RED}choice must be in range 1...{len(pathDict)}{C.WHITE}')

